#### 1. Libraries needed

In [2]:
import os
import pandas as pd

In [3]:
cwd = os.getcwd()
cwd

'C:\\Users\\paola\\Documents\\ironhack\\190712-pandas-project\\your-code'

#### 2. How is data coming? Making sense of the columns
**Site** - Site code: It is necessary since I want to merge two areas from Lewisham. *Keep it.* 

**Species** - pollutants names with standard chemical notation (e.g., CO = carbon monoxide), meteorological parameters are as follows: WSPD = wind speed, WDIR = wind direction, BP = barometric pressure, TMP = ambient temperature, RAIN = rainfall, SOLR = incoming solar radiation. *Keep it.*

**DateTime** - Date and time of the measurement. This data is hourly mean value, i.e. for 01-jan-2018 12:00 encompasses measurements taken between 12:00 and 12:59 on 01 January 2018. *Keep it.*

**Value** - Measurement concentration. *Keep it.*

**Units** - Shows the units the values are quoted in. *Keep it.*

**Provisional or Ratified** - P for provisional and R for Ratified. Provisional measurements are subjected to change. Ratified measurements have been through the full QAQC procedure and will not change in the future. *Keep it.*

#####   **Examining data for potential issues**

In [29]:
#First source
read_lewisham_HP1 = pd.read_csv('Lewisham_HP1_raw.csv') 

display(read_lewisham_HP1.info())
display(read_lewisham_HP1.tail(3))
print(type(read_lewisham_HP1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34848 entries, 0 to 34847
Data columns (total 6 columns):
Site                       34848 non-null object
Species                    34848 non-null object
ReadingDateTime            34848 non-null object
Value                      28947 non-null float64
Units                      34848 non-null object
Provisional or Ratified    34848 non-null object
dtypes: float64(1), object(5)
memory usage: 1.6+ MB


None

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
34845,HP1,TSP,30/06/2019 21:00,7.1,ug/m3,P
34846,HP1,TSP,30/06/2019 22:00,6.8,ug/m3,P
34847,HP1,TSP,30/06/2019 23:00,6.1,ug/m3,P


<class 'pandas.core.frame.DataFrame'>


In [30]:
#Second source
read_Lewisham_LW2 = pd.read_csv('Lewisham_LW2_raw.csv')

display(read_Lewisham_LW2.info())
display(read_Lewisham_LW2.tail(3))
print(type(read_Lewisham_LW2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29040 entries, 0 to 29039
Data columns (total 6 columns):
Site                       29040 non-null object
Species                    29040 non-null object
ReadingDateTime            29040 non-null object
Value                      17307 non-null float64
Units                      29040 non-null object
Provisional or Ratified    29040 non-null object
dtypes: float64(1), object(5)
memory usage: 1.3+ MB


None

,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
29037,LW1,SO2,30/06/2019 21:00,NaN,ug m-3,P
29038,LW1,SO2,30/06/2019 22:00,NaN,ug m-3,P
29039,LW1,SO2,30/06/2019 23:00,NaN,ug m-3,P


<class 'pandas.core.frame.DataFrame'>


#### 3. Data Cleaning
#####   **3.1 Changing title names**

In [31]:
#First source
read_lewisham_HP1 = read_lewisham_HP1.rename(columns = {"Species":"Pollutant Name","ReadingDateTime":"Reading Time"})
display(read_lewisham_HP1.head(1))

,Site,Pollutant Name,Reading Time,Value,Units,Provisional or Ratified
0,HP1,NO,01/11/2018 00:00,NaN,ug m-3,P


In [32]:
#Second source
read_Lewisham_LW2 = read_Lewisham_LW2.rename(columns = {"Species":"Pollutant Name","ReadingDateTime":"Reading Time"})
display(read_Lewisham_LW2.head(1))

,Site,Pollutant Name,Reading Time,Value,Units,Provisional or Ratified
0,LW1,NO,01/11/2018 00:00,13.9,ug m-3,R


#####   **3.2 Rearranging columns order**

In [33]:
#First source
new_order = ['Site','Reading Time','Pollutant Name','Value','Units','Provisional or Ratified']
lewishamHP1_df = read_lewisham_HP1[new_order]
lewishamHP1_df.head(1)

,Site,Reading Time,Pollutant Name,Value,Units,Provisional or Ratified
0,HP1,01/11/2018 00:00,NO,NaN,ug m-3,P


In [34]:
#Second source
new_order = ['Site','Reading Time','Pollutant Name','Value','Units','Provisional or Ratified']
lewishamLW2_df = read_Lewisham_LW2[new_order]
lewishamLW2_df.head(1)

,Site,Reading Time,Pollutant Name,Value,Units,Provisional or Ratified
0,LW1,01/11/2018 00:00,NO,13.9,ug m-3,R


#####   **3.3 Changing names**

In [35]:
#Source 1
lewishamHP1_df['Pollutant Name'].value_counts()

NO2      5808
PM10     5808
TSP      5808
NO       5808
PM2.5    5808
O3       5808
Name: Pollutant Name, dtype: int64

In [36]:
lewishamHP1_df = lewishamHP1_df.replace({'Pollutant Name':'TSP'},{'Pollutant Name':'Total Suspended Particles'},regex=True)

In [27]:
#Challenge: unexpected changes of the names. I was thinking to replace the 'NO', but it changed me the 'NO2' as well. And will
#change the 'NOX' from the second source. So I prefered not to change it, to keep recognized abreviated names from the pollutants. 
#This kind of situations is one sample of the precautions when we decide to change values or names.

In [37]:
#Source 2
lewishamLW2_df['Site'].value_counts()

LW1    29040
Name: Site, dtype: int64

In [39]:
#Replacing LW1 to LW2
lewishamLW2_df = lewishamLW2_df.replace({'Site':'LW1'},{'Site':'LW2'},regex=True)

In [40]:
lewishamLW2_df['Pollutant Name'].value_counts()

SO2    5808
NO2    5808
NO     5808
NOX    5808
O3     5808
Name: Pollutant Name, dtype: int64

#####   **3.4 Skiping NaN values**

In [41]:
#Source 1
null_values = lewishamHP1_df.isnull().sum()
null_values

Site                          0
Reading Time                  0
Pollutant Name                0
Value                      5901
Units                         0
Provisional or Ratified       0
dtype: int64

In [47]:
#Droping the rows that had NaN values on its 'Value' column
lewishamHP1_df = lewishamHP1_df.dropna(subset=['Value'])

null_values = lewishamHP1_df.isnull().sum()
null_values

Site                       0
Reading Time               0
Pollutant Name             0
Value                      0
Units                      0
Provisional or Ratified    0
dtype: int64

In [49]:
#Source 2
null_values2 = lewishamLW2_df.isnull().sum() 
#There were 11733 NaN values
lewishamLW2_df = lewishamLW2_df.dropna(subset=['Value'])

#####   **3.5 Combining Data Frames**

In [51]:
#Using 'concat' method to attach rows with the same number of columns
combined_PH1_LW2 = pd.concat([lewishamHP1_df,lewishamLW2_df],axis = 0)

display(combined_PH1_LW2['Pollutant Name'].value_counts())
display(combined_PH1_LW2.info())

NO2                          10459
NO                           10197
NOX                           5769
Total Suspended Particles     5133
PM10                          5133
PM2.5                         4865
O3                            4698
Name: Pollutant Name, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46254 entries, 1119 to 17423
Data columns (total 6 columns):
Site                       46254 non-null object
Reading Time               46254 non-null object
Pollutant Name             46254 non-null object
Value                      46254 non-null float64
Units                      46254 non-null object
Provisional or Ratified    46254 non-null object
dtypes: float64(1), object(5)
memory usage: 2.5+ MB


None

#####   **3.6 Checking extreme values or outliers (negative numbers)**

In [67]:
stats = combined_PH1_LW2.describe().transpose()
stats

,count,mean,std,min,25%,50%,75%,max
Value,46254.0,31.164157,37.216223,-3.7,9.0,19.4,42.0,710.5


In [59]:
#There are negative values: not reliable because of possible uncalibrated instruments

test = combined_PH1_LW2[(combined_PH1_LW2['Value'] <= 0)]

#Output --> There are 87 records: test[['Reading Time','Pollutant Name','Value','Units','Provisional or Ratified']]

In [60]:
#I'll just note on the column 'Provisional or Ratified column that these values are not reliable
combined_PH1_LW2.loc[(combined_PH1_LW2['Value'] <= 0),'Provisional or Ratified'] = 'P - not reliable data'

In [65]:
# Checking test note
test = combined_PH1_LW2[(combined_PH1_LW2['Value'] <= 0)]
test[['Reading Time','Pollutant Name','Value','Units','Provisional or Ratified']].head(1)

,Reading Time,Pollutant Name,Value,Units,Provisional or Ratified
2356,07/02/2019 04:00,NO,-0.3,ug m-3,P - not reliable data


#####   Checking extreme values or outliers (maximun points)

In [69]:
# Identifying outliers with the interquartile range (IQR). 
# Any values outside this range are potential outliers and should be investigated.

stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
Value,46254.0,31.164157,37.216223,-3.7,9.0,19.4,42.0,710.5,33.0


#####   **3.7 Creating an empty df called 'outliers'**

In [86]:
outliers = pd.DataFrame(columns=combined_PH1_LW2.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = combined_PH1_LW2[(combined_PH1_LW2[col] < lower)|(combined_PH1_LW2[col] > upper)].copy()
    results['Outlier']  = col
    outliers = outliers.append(results)
    
display(outliers.head(3))
# 2075 outlier entries

# This is an empty dataframe called 'outliers' with the same columns as the core data frame 'combined_PH1_LW2'. 
# It could be investigated further and find out if they should be kept in the data or dropped. 

,Outlier,Pollutant Name,Provisional or Ratified,Reading Time,Site,Units,Value
1296,Value,NO,P,25/12/2018 00:00,HP1,ug m-3,131.20000
1297,Value,NO,P,25/12/2018 01:00,HP1,ug m-3,142.39999
1298,Value,NO,P,25/12/2018 02:00,HP1,ug m-3,125.20000
1299,Value,NO,P,25/12/2018 03:00,HP1,ug m-3,102.50000
1354,Value,NO,P,27/12/2018 10:00,HP1,ug m-3,102.80000


In [87]:
# Exporting outliers df for further analysis
outliers.to_csv('Lewisham_outliers.csv',index=False)

#####   **3.8 Exporting clean data**

In [89]:
display(combined_PH1_LW2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46254 entries, 1119 to 17423
Data columns (total 6 columns):
Site                       46254 non-null object
Reading Time               46254 non-null object
Pollutant Name             46254 non-null object
Value                      46254 non-null float64
Units                      46254 non-null object
Provisional or Ratified    46254 non-null object
dtypes: float64(1), object(5)
memory usage: 3.7+ MB


None

In [90]:
combined_PH1_LW2.to_csv('Lewisham_combined_clean.csv',index=False)